In [1]:
import boto3

In [2]:
AWS_DEFAULT_REGION="us-west-2"
AWS_ACCESS_KEY_ID=""
AWS_SECRET_ACCESS_KEY=""
AWS_SESSION_TOKEN=""

In [3]:
bedrock_client = boto3.client(
    'bedrock-runtime',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name='us-west-2'
)

In [4]:
from langchain_aws import ChatBedrock
from langchain.embeddings import BedrockEmbeddings
from langchain_pinecone import PineconeVectorStore

/Users/thej/miniforge3/envs/12labshack/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
chat = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs={"temperature": 0.1},
    client=bedrock_client,
    streaming=True,
)

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0",
                                       client=bedrock_client)

In [6]:
index_name = "12labshack"
namespace = "ns1"
vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=bedrock_embeddings,
    namespace=namespace,
    pinecone_api_key='103636ad-271b-4e3c-bf2e-ed594c2b165e'
)

In [7]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [8]:
retriever = vectorstore.as_retriever()

In [9]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    chat, retriever, contextualize_q_prompt
)

In [10]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(chat, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [19]:
from langchain_core.messages import HumanMessage,AIMessage

chat_history = []

question = "Is deadpool action movie?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(ai_msg_1["answer"])])

In [20]:
ai_msg_1["answer"]

'Based on the provided context, yes, Deadpool is an action movie franchise that blends intense action sequences with humor and irreverent comedy.\n\nThe context mentions that the final chapter brings the narrative to its "climax, with Deadpool and his team engaging in the anticipated action sequence." It describes "rapid succession of scenes featuring combat, explosions, and high-speed chases" which encapsulates "the essence of the Deadpool franchise—action-packed sequences infused with humor and irreverence."\n\nAdditionally, one of the earlier chapters refers to a "big slow-motion action sequence" that is expected in superhero films, setting up the audience\'s anticipation for major action set pieces.\n\nSo while the Deadpool movies incorporate a lot of humor and meta-commentary, they are fundamentally action movies at their core, featuring over-the-top action sequences that are a staple of the franchise. The action is simply presented through an irreverent and comedic lens that sets

In [21]:
second_question = "Tell me more about it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Here are some additional details about the action elements in the Deadpool movies:

The action sequences are highly stylized and exaggerated, fitting with Deadpool's over-the-top personality. They feature excessive violence and gore, but played for dark comedic effect rather than taking themselves too seriously.

Deadpool's fighting style incorporates a mix of martial arts, gunplay, swordplay, and creative use of any weapons or objects at his disposal. The choreography is fast-paced and acrobatic.

The movies revel in breaking the fourth wall and making meta-references about action movie clichés, while simultaneously indulging in those very clichés like slow-motion shots and one-liners during fights.

The action is intercut with Deadpool's trademark raunchy humor and pop culture references, preventing the tone from ever getting too grim or self-serious.

Despite the comedic elements, the stunt work and visual effects are top-notch, creating slick, high-octane set pieces on par with maj